<a href="https://colab.research.google.com/drive/1tx5iDzZix9Wi2Ma1xTXYkUJ1fY8gwcIU#scrollTo=esOiLvqPMb2E">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!git clone https://github.com/Fonsydo/Proyecto_IA.git

Cloning into 'Proyecto_IA'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 73 (delta 23), reused 37 (delta 9), pack-reused 0
Unpacking objects: 100% (73/73), 2.86 MiB | 2.89 MiB/s, done.
Filtering content: 100% (7/7), 900.04 MiB | 7.67 MiB/s, done.


# Inteligencia Artificial para las Ciencias e Ingenierías
## Proyecto Sistema de Recomendación de Películas
### Miembros del grupo
* Fonsy Johan Mercado Agudelo, CC 1020472932, Ingeniería eléctrica
* Orlando José Salazar Polo, CC 1152714311, Ingeniería eléctrica
* Angie Dayana Rincón Mandón, CC 1091681348, Ingeniería eléctrica

### Exploración de datos


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import linear_model
from wordcloud import WordCloud

### The movies dataset
Para el desarrollo de este proyecto se seleccionó el dataset [The Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) proporcionado en kaggle, estos archivos contienen metadatos de 45.000 películas enumeradas en el conjunto de datos completo de MovieLens. Este conjunto de datos también tiene archivos que contienen 26 millones de calificaciones de 270.000 usuarios para las 45.000 películas. Las calificaciones están en una escala de 1 a 5 y se han obtenido del sitio web oficial de GroupLens.

El dataset tiene un total de 900 MB y contiene los siguientes archivos .csv

- **movies_metadata.csv**: Archivo principal de metadatos de películas, contiene información de 45000 películas incluyendo carteles, fondos, presupuesto, ingresos, fechas de lanzamiento, idiomas, países de producción y empresas.

- **keywords.csv**: Contiene las palabras clave de la trama de la película en forma de un objeto JSON en cadena.

- **credits.csv**: Contiene información sobre el reparto y equipo técnico de las películas en forma de objeto JSON en cadena.

- **links.csv**: Contiene los ID de TMDB e IMDB de las películas que aparecen en el conjunto de datos de MovieLens.

- **links_small.csv**: Contiene los ID de TMDB e IMDB de un subconjunto de 9.000 películas del conjunto de datos completo.

- **ratings_small.csv** El subconjunto de 100.000 calificaciones de 700 usuarios en 9.000 películas

In [ ]:
credits = pd.read_csv('/content/Proyecto_IA/the-movies-dataset/credits.csv')
keywords = pd.read_csv('/content/Proyecto_IA/the-movies-dataset/keywords.csv')
movies = pd.read_csv('/content/Proyecto_IA/the-movies-dataset/movies_metadata.csv', low_memory=False).\
                        drop(['belongs_to_collection', 'homepage', 'imdb_id', 'poster_path', 'status', 'title', 'video'], axis=1)

movies['id'] = movies['id'].apply(pd.to_numeric, errors='coerce')
movies.dropna(inplace=True)
movies['id'] = movies['id'].astype('int64')

df = movies.merge(keywords, on='id'). merge(credits, on='id')

df['original_language'] = df['original_language'].fillna('')
df['runtime'] = df['runtime'].fillna(0)
df['tagline'] = df['tagline'].fillna('')

df.dropna(inplace=True)

In [ ]:
df.sample(2)

,adult,budget,genres,id,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,keywords,cast,crew
19866,False,0,"[{'id': 37, 'name': 'Western'}]",31390,en,Brothers in Arms,Old West. New Soul. Big Payback.,1.061451,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2005-01-01,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",A revisionist western,2.7,5.0,"[{'id': 155291, 'name': 'cowboy'}, {'id': 1889...","[{'cast_id': 1, 'character': 'Driscoll', 'cred...","[{'credit_id': '52fe44799251416c9101200b', 'de..."
1934,False,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31342,en,The Love Letter,A romantic comedy about a mysterious love lett...,2.012283,"[{'name': 'DreamWorks SKG', 'id': 27}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1999-05-21,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",No one knows who sent it. No one knows who it'...,5.9,17.0,"[{'id': 3452, 'name': 'love letter'}, {'id': 1...","[{'cast_id': 1, 'character': 'Helen', 'credit_...","[{'credit_id': '52fe44769251416c910117ed', 'de..."


In [ ]:
def get_text(text, obj='name'):
    text = eval(text)
    
    if len(text) == 1:
        for i in text:
            return i[obj]
    else:
        s = []
        for i in text:
            s.append(i[obj])
        return ', '.join(s)
    
df['genres'] = df['genres'].apply(get_text)
df['production_companies'] = df['production_companies'].apply(get_text)
df['production_countries'] = df['production_countries'].apply(get_text)
df['crew'] = df['crew'].apply(get_text)
df['spoken_languages'] = df['spoken_languages'].apply(get_text)
df['keywords'] = df['keywords'].apply(get_text)

df['characters'] = df['cast'].apply(get_text, obj='character')
df['actors'] = df['cast'].apply(get_text)

df['release_date'] = pd.to_datetime(df['release_date'])
df['budget'] = df['budget'].astype('float64')
df['popularity'] = df['popularity'].astype('float64')

df.drop('cast', axis=1, inplace=True)
df = df[~df['original_title'].duplicated()]
df = df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df_plot = df[(df['revenue']!=0) & (df['budget']!=0)]

regr = linear_model.LinearRegression()

plt.figure(figsize=(8,5))
plt.suptitle('The Influence of Budget and Revenue\non Popularity of Movies', fontsize=14, weight=600, color='#333d29')

for i, col in enumerate(['budget', 'revenue']):
    regr.fit(np.array(df_plot[col]).reshape(-1,1), df_plot['popularity'].values)
    plt.subplot(2,1,i+1)
    plt.grid()
    plt.scatter(df_plot[col], df_plot['popularity'], alpha=.4, s=100, color='#06837f')
    plt.plot(df_plot[col], df_plot[col]*regr.coef_[0]+regr.intercept_, color='#fed811')
    plt.xlabel(col)
    plt.ylabel('popularity')
plt.tight_layout(pad=0.7);

In [ ]:
plt.figure(figsize=(20,20))
plt.title('The Most Common Word in Movie Overviews\n', fontsize=30, weight=600, color='#333d29')
wc = WordCloud(max_words=1000, min_font_size=10,
                height=700,width=1500,background_color="white").generate(' '.join(df['overview']))
plt.axis('off')
plt.imshow(wc)

In [ ]:
genres_list = []
for i in df['genres']:
    genres_list.extend(i.split(', '))

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14,6))

df_plot = pd.DataFrame(Counter(genres_list).most_common(5), columns=['genre', 'total'])
ax = sns.barplot(data=df_plot, x='genre', y='total', ax=axes[0], palette=['#06837f', '#02cecb', '#b4ffff', '#f8e16c', '#fed811'])
ax.set_title('Top 5 Genres in Movies', fontsize=18, weight=600, color='#333d29')
sns.despine()

df_plot_full = pd.DataFrame([Counter(genres_list)]).transpose().sort_values(by=0, ascending=False)
df_plot.loc[len(df_plot)] = {'genre': 'Others', 'total':df_plot_full[6:].sum()[0]}
plt.title('Percentage Ratio of Movie Genres', fontsize=18, weight=600, color='#333d29')
wedges, texts, autotexts = axes[1].pie(x=df_plot['total'], labels=df_plot['genre'], autopct='%.2f%%',
                                       textprops=dict(fontsize=14), explode=[0.1,0,0,0,0,0], colors=['#06837f', '#02cecb', '#b4ffff', '#f8e16c', '#fed811', '#fdc100'])

for autotext in autotexts:
    autotext.set_color('#1c2541')
    autotext.set_weight('bold')

axes[1].axis('off')

In [ ]:
sns.displot(data=df, x='release_date', kind='hist', kde=True,
            color='#fdc100', facecolor='#06837f', edgecolor='#64b6ac', line_kws={'lw': 3}, aspect=3)
plt.title('Total Released Movie by Date', fontsize=18, weight=600, color='#333d29')

In [ ]:
sns.relplot(data=df, x='vote_average', y='popularity', size='vote_count',
            sizes=(20, 200), alpha=.5, aspect=2, color='#06837f')
plt.title('The Relationship Between Rating and Popularity', fontsize=15, weight=600, color='#333d29')

In [ ]:
df_plot = pd.DataFrame(Counter(genres_list).most_common(5), columns=['genre', 'total'])
df_plot = df[df['genres'].isin(df_plot['genre'].to_numpy())]

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10,6))

plt.suptitle('Data Distribution Across Top 5 Genres', fontsize=18, weight=600, color='#333d29')
for i, y in enumerate(['runtime', 'popularity', 'budget', 'revenue']):
    sns.stripplot(data=df_plot, x='genres', y=y, ax=axes.flatten()[i], palette=['#06837f', '#02cecb', '#b4ffff', '#f8e16c', '#fed811'])

plt.tight_layout()

In [ ]:
plt.figure(figsize=(12,10))
plt.title('Correlation of Movie Features\n', fontsize=18, weight=600, color='#333d29')
sns.heatmap(df.corr(), annot=True, cmap=['#004346', '#036666', '#06837f', '#02cecb', '#b4ffff', '#f8e16c', '#fed811', '#fdc100'])